In [5]:
import pandas as pd
import numpy as np
import yfinance as yf
from pypfopt import risk_models

from pypfopt.efficient_frontier import EfficientFrontier
from datetime import date, datetime, timedelta
from arch import arch_model
from pypfopt import expected_returns


In [100]:
len(data_window)

31

In [98]:
cov = risk_models.sample_cov(data_window)
cov


,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN,DIS,MCD,NEE,BA,CAT,XOM,CVX,RIO,BHP
AAPL,0.838870,0.857991,0.958800,0.538758,0.534805,0.927365,0.914705,0.467837,0.757533,0.696359,0.704185,1.066454,0.660200,0.697756,0.926070,0.796676,0.827410
MSFT,0.857991,0.954585,1.064311,0.567929,0.560692,0.996994,0.971317,0.517079,0.777417,0.722887,0.750246,1.081001,0.688979,0.738023,0.997280,0.830722,0.891934
NVDA,0.958800,1.064311,1.398754,0.572341,0.634334,1.105136,1.133450,0.586921,0.932872,0.904362,0.861310,1.370197,0.794553,0.929341,1.223403,0.988507,1.094934
JNJ,0.538758,0.567929,0.572341,0.528062,0.388002,0.619029,0.566449,0.296441,0.467825,0.440409,0.585965,0.614929,0.485569,0.441529,0.612592,0.535074,0.550467
NVS,0.534805,0.560692,0.634334,0.388002,0.436746,0.624358,0.614092,0.317908,0.515376,0.462269,0.508130,0.773903,0.471850,0.494334,0.619764,0.549998,0.594969
JPM,0.927365,0.996994,1.105136,0.619029,0.624358,1.271996,1.187893,0.496628,0.922264,0.878668,0.801132,1.428549,0.789520,0.868132,1.333793,1.014157,1.105876
GS,0.914705,0.971317,1.133450,0.566449,0.614092,1.187893,1.235838,0.497680,1.001696,0.914644,0.817488,1.437471,0.774850,0.866014,1.275050,1.004444,1.090211
AMZN,0.467837,0.517079,0.586921,0.296441,0.317908,0.496628,0.497680,0.353775,0.425131,0.335743,0.386289,0.506517,0.393050,0.383774,0.418726,0.427455,0.508205
DIS,0.757533,0.777417,0.932872,0.467825,0.515376,0.922264,1.001696,0.425131,0.950263,0.770044,0.662708,1.220730,0.699419,0.738551,1.024610,0.837685,0.911738
MCD,0.696359,0.722887,0.904362,0.440409,0.462269,0.878668,0.914644,0.335743,0.770044,0.945533,0.671675,1.134603,0.587901,0.718773,1.127212,0.770274,0.831442


In [64]:

# replace diagonal elements of cov matrix by GARCH-predicted variance.
# GARCH_var = GARCH_predict(symbol_list, start, end, interval)
cov =  risk_models.sample_cov(data_window)
adjust_cov_matrix =  cov.copy()
for symbols in stocks:
    adjust_cov_matrix[symbols][symbols] = pred[symbols]

In [66]:
cov

,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN,DIS,MCD,NEE,BA,CAT,XOM,CVX,RIO,BHP
AAPL,0.058274,0.018206,0.019168,0.010635,0.010151,0.015689,0.015532,0.018577,0.013300,0.010370,0.006528,0.016455,0.016884,0.012073,0.012828,0.016721,0.017755
MSFT,0.018206,0.062191,0.025893,0.013602,0.012105,0.021510,0.020994,0.027030,0.017956,0.014579,0.011996,0.015645,0.019634,0.016352,0.017380,0.019693,0.024652
NVDA,0.019168,0.025893,0.080974,0.012013,0.012549,0.023663,0.023516,0.023010,0.021142,0.012394,0.009928,0.016939,0.017510,0.017696,0.018298,0.020796,0.020378
JNJ,0.010635,0.013602,0.012013,0.022835,0.010566,0.015462,0.015572,0.015778,0.013759,0.012092,0.011097,0.014879,0.011858,0.013271,0.013140,0.012845,0.014146
NVS,0.010151,0.012105,0.012549,0.010566,0.026887,0.012448,0.013018,0.012610,0.013560,0.008817,0.009904,0.011366,0.008999,0.010747,0.011320,0.016020,0.016725
JPM,0.015689,0.021510,0.023663,0.015462,0.012448,0.038988,0.031055,0.022781,0.019888,0.013637,0.011010,0.019604,0.018901,0.019105,0.020044,0.021848,0.021244
GS,0.015532,0.020994,0.023516,0.015572,0.013018,0.031055,0.038817,0.024616,0.020239,0.013608,0.010162,0.020024,0.019901,0.018880,0.019726,0.022448,0.021558
AMZN,0.018577,0.027030,0.023010,0.015778,0.012610,0.022781,0.024616,0.098391,0.023871,0.015277,0.009997,0.021376,0.018288,0.015854,0.017691,0.020650,0.025167
DIS,0.013300,0.017956,0.021142,0.013759,0.013560,0.019888,0.020239,0.023871,0.038648,0.013231,0.010704,0.018475,0.013951,0.014461,0.014461,0.017491,0.017818
MCD,0.010370,0.014579,0.012394,0.012092,0.008817,0.013637,0.013608,0.015277,0.013231,0.024089,0.009396,0.012998,0.012230,0.012304,0.011611,0.013629,0.014545


In [65]:
adjust_cov_matrix

,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN,DIS,MCD,NEE,BA,CAT,XOM,CVX,RIO,BHP
AAPL,0.042332,0.018206,0.019168,0.010635,0.010151,0.015689,0.015532,0.018577,0.013300,0.010370,0.006528,0.016455,0.016884,0.012073,0.012828,0.016721,0.017755
MSFT,0.018206,0.031458,0.025893,0.013602,0.012105,0.021510,0.020994,0.027030,0.017956,0.014579,0.011996,0.015645,0.019634,0.016352,0.017380,0.019693,0.024652
NVDA,0.019168,0.025893,0.048492,0.012013,0.012549,0.023663,0.023516,0.023010,0.021142,0.012394,0.009928,0.016939,0.017510,0.017696,0.018298,0.020796,0.020378
JNJ,0.010635,0.013602,0.012013,0.012198,0.010566,0.015462,0.015572,0.015778,0.013759,0.012092,0.011097,0.014879,0.011858,0.013271,0.013140,0.012845,0.014146
NVS,0.010151,0.012105,0.012549,0.010566,0.016388,0.012448,0.013018,0.012610,0.013560,0.008817,0.009904,0.011366,0.008999,0.010747,0.011320,0.016020,0.016725
JPM,0.015689,0.021510,0.023663,0.015462,0.012448,0.039175,0.031055,0.022781,0.019888,0.013637,0.011010,0.019604,0.018901,0.019105,0.020044,0.021848,0.021244
GS,0.015532,0.020994,0.023516,0.015572,0.013018,0.031055,0.046471,0.024616,0.020239,0.013608,0.010162,0.020024,0.019901,0.018880,0.019726,0.022448,0.021558
AMZN,0.018577,0.027030,0.023010,0.015778,0.012610,0.022781,0.024616,0.035347,0.023871,0.015277,0.009997,0.021376,0.018288,0.015854,0.017691,0.020650,0.025167
DIS,0.013300,0.017956,0.021142,0.013759,0.013560,0.019888,0.020239,0.023871,0.023975,0.013231,0.010704,0.018475,0.013951,0.014461,0.014461,0.017491,0.017818
MCD,0.010370,0.014579,0.012394,0.012092,0.008817,0.013637,0.013608,0.015277,0.013231,0.008839,0.009396,0.012998,0.012230,0.012304,0.011611,0.013629,0.014545


## 
for monthly re-balancing, i should be using the 1 month ahead predicted volatility.
1. investigate the GARCH scaling
2. how i handle the weekly resampling in the predictions
3. Ensure correct volatility for the frequency of interest.
   
 $VOL=  \sigma \sqrt{T}$
+ v = volatility over some interval of time
+ σ =standard deviation of returns
+ T = number of periods in the time horizon

volatiltiy = variance^{1/2} = standard deviation

### Testing GARCH forecasts

In [5]:
from datetime import date, datetime, timedelta
from arch import arch_model

In [44]:
start_date = '2009-01-01' 
end = '2013-01-01'
returns = yf.download(stocks, start_date, end).pct_change()['Adj Close'].dropna()

[*********************100%***********************]  17 of 17 completed


In [45]:
Data = pd.DataFrame()
Data['Daily_Returns'] = returns['AAPL'].copy()

In [47]:
def FIG_GARCH(Data):
    AR_Data=Data['Daily_Returns']*100
    FIGARCH11 = arch_model(AR_Data, dist ='t', vol="FIGARCH")
    res = FIGARCH11.fit(update_freq=5)
    forecasts = res.forecast(horizon=2, method='simulation', simulations=1000)
    print(forecasts.variance.dropna().tail())
    
FIG_GARCH(Data)

    

Iteration:      5,   Func. Count:     42,   Neg. LLF: 1985.5705029522974
Iteration:     10,   Func. Count:     83,   Neg. LLF: 1975.3577392473703
Iteration:     15,   Func. Count:    118,   Neg. LLF: 1975.1717124393579
Iteration:     20,   Func. Count:    152,   Neg. LLF: 1975.1711981798612
Optimization terminated successfully    (Exit mode 0)
            Current function value: 1975.1711981798628
            Iterations: 20
            Function evaluations: 152
            Gradient evaluations: 20
                 h.1       h.2
Date                          
2012-12-31  5.018104  4.472138


/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


In [48]:
#Fitting of FIGARCH11(1,1)
def FIGARCH_Model_Student(Data,hor=1):
    AR_Data=Data['Daily_Returns']*100
    FIGARCH11 = arch_model(AR_Data, dist ='t', vol="FIGARCH")
    res_FIGARCH11 = FIGARCH11.fit(disp='off', update_freq=5)
    CV_FIGARCH11 = res_FIGARCH11.conditional_volatility
    For_CV_FIGARCH11 = res_FIGARCH11.forecast(horizon=1, method='simulation', simulations=1000).variance.dropna()
    return FIGARCH11, res_FIGARCH11, CV_FIGARCH11, For_CV_FIGARCH11

In [49]:
FIGARCH11, res_FIGARCH11, CV_FIGARCH11, For_CV_FIGARCH11 = FIGARCH_Model_Student(Data,1)


/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


In [79]:
For_CV_FIGARCH11/100


0.045545036114844686

In [80]:
np.var(Data)

Daily_Returns    0.000336
dtype: float64

In [50]:
#Fitting of GARCH(1,1)
def GARCH_Model_Student (Data):
    AR_Data=Data['Daily_Returns']*100
    GARCH11 = arch_model(AR_Data, dist ='t')
    res_GARCH11 = GARCH11.fit(disp='off')
    CV_GARCH11 = res_GARCH11.conditional_volatility
    For_CV_GARCH11 = np.array(res_GARCH11.forecast(horizon=1).variance.dropna())[0][0]
    return GARCH11, res_GARCH11, CV_GARCH11, For_CV_GARCH11


In [54]:
FIGARCH11, res_FIGARCH11, CV_FIGARCH11, For_CV_FIGARCH11 = GARCH_Model_Student(Data)

/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


### Attempt to use our weekly forecasts.
+ I think we need to address teh frequency

In [ ]:
# Define the tickers of the stocks you're interested in
tickers = ['AAPL', 'MSFT', 'NVDA', 'JNJ', 'NVS','JPM','GS','AMZN','DIS','MCD','NEE','BA','CAT','XOM','CVX','RIO','BHP']

# Define the start and end dates for the data
start_date = '2009-01-01' 
end_date = '2022-12-31'

# Initialize an empty DataFrame to store the results
volatility_df = pd.DataFrame()

# Loop over each ticker
for ticker in tickers:
    # Download the stock price data
    data = yf.download(ticker, start=start_date, end=end_date)
    
    # Resample the data to a weekly frequency and calculate the weekly returns
    data = data.resample('W').last()
    data['returns'] = data['Adj Close'].pct_change()
    
    # Drop missing values
    data = data.dropna()

    # Define the GARCH model
    model = arch_model(data['returns'], vol='Garch', p=1, q=1)

    # Fit the GARCH model
    res = model.fit(disp='off')

    # Forecast the volatility for the next 4 weeks
    forecasts = res.forecast(start=0, horizon=4)

    # Add the forecasted volatility to the DataFrame
    for i in range(1, 5):
        volatility_df[ticker + '_week_' + str(i)] = np.sqrt(forecasts.variance['h.' + str(i)])

# Drop missing values from the DataFrame
volatility_df = volatility_df.dropna()

# Print the DataFrame
print(volatility_df)


In [6]:
# Define the tickers of the stocks you're interested in
tickers = ['AAPL', 'MSFT', 'NVDA', 'JNJ', 'NVS','JPM','GS','AMZN','DIS','MCD','NEE','BA','CAT','XOM','CVX','RIO','BHP']

# Define the start and end dates for the data
start_date = '2009-01-01' 
end_date = '2022-12-31'

# Initialize an empty DataFrame to store the results
volatility_df = pd.DataFrame()

# Loop over each ticker
for ticker in tickers:
    # Download the stock price data
    data = yf.download(ticker, start=start_date, end=end_date)
    
    # Resample the data to a weekly frequency and calculate the weekly returns
    data = data.resample('W').last()
    data['returns'] = data['Adj Close'].pct_change()
    
    # Drop missing values
    data = data.dropna()

    # Define the GARCH model
    model = arch_model(data['returns'], vol='Garch', p=1, q=1)

    # Fit the GARCH model
    res = model.fit(disp='off')

    # Forecast the volatility for the next 4 weeks
    forecasts = res.forecast(start=0, horizon=4)

    # Add the forecasted volatility for the 4th week to the DataFrame
    forecasted_volatility = pd.Series(np.sqrt(forecasts.variance['h.4']), index=data.index)*np.sqrt(252) 
    
    # Shift the index by 4 weeks
    forecasted_volatility.index = forecasted_volatility.index + pd.DateOffset(weeks=4)

    # Add the forecasted volatility to the DataFrame
    volatility_df[ticker] = forecasted_volatility

# Drop missing values from the DataFrame
volatility_df = volatility_df.dropna()

# Print the DataFrame
print(volatility_df)


[*********************100%***********************]  1 of 1 completed


/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001598. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


[*********************100%***********************]  1 of 1 completed


/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001121. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


[*********************100%***********************]  1 of 1 completed


...

/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.001341. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


[*********************100%***********************]  1 of 1 completed


/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.002726. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


[*********************100%***********************]  1 of 1 completed
                AAPL      MSFT      NVDA       JNJ       NVS       JPM  \
Date                                                                     
2009-02-08  0.721720  0.817985  1.462632  0.395513  0.524601  1.955287   
2009-02-15  0.800274  0.755982  1.424456  0.386940  0.506544  1.827627   
2009-02-22  0.789261  0.903154  1.394272  0.379075  0.510183  1.597926   
2009-03-01  0.730265  0.831021  1.360456  0.373584  0.557409  1.396670   
2009-03-08  0.805159  0.986027  1.516998  0.354987  0.534762  1.295375   
...              ...       ...       ...       ...       ...       ...   
2023-01-01  0.667169  0.635580  1.160936  0.335674  0.407172  0.620608   
2023-01-08  0.665970  0.626657  1.133667  0.335967  0.400830  0.616297   
2023-01-15  0.688264  0.592654  1.112416  0.326186  0.395299  0.612507   
2023-01-22  0.662141  0.576946  1.123845  0.320855  0.389864  0.592901   
2023-01-29  0.639226  0.549698  1.107949  0

/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.002203. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/thesis_2/lib/python3.10/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


In [7]:
volatility_df   

,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN,DIS,MCD,NEE,BA,CAT,XOM,CVX,RIO,BHP
Date,,,,,,,,,,,,,,,,,
2009-02-08,0.721720,0.817985,1.462632,0.395513,0.524601,1.955287,1.180200,1.014781,0.784246,0.439960,0.596129,1.013027,1.193984,0.617250,0.705546,1.759818,1.130731
2009-02-15,0.800274,0.755982,1.424456,0.386940,0.506544,1.827627,1.247614,1.015741,0.751180,0.396939,0.533388,0.991368,1.158823,0.586353,0.661846,1.748356,1.130784
2009-02-22,0.789261,0.903154,1.394272,0.379075,0.510183,1.597926,1.117247,0.989905,0.725254,0.391264,0.545091,0.944989,1.174570,0.557168,0.621611,1.694593,1.119544
2009-03-01,0.730265,0.831021,1.360456,0.373584,0.557409,1.396670,1.070177,1.082140,0.679348,0.369528,0.652679,0.899079,1.273180,0.540050,0.580753,1.639088,1.086229
2009-03-08,0.805159,0.986027,1.516998,0.354987,0.534762,1.295375,1.370726,1.123430,0.694537,0.358790,0.580054,0.862102,1.217552,0.578768,0.645339,2.090917,1.274180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-01,0.667169,0.635580,1.160936,0.335674,0.407172,0.620608,0.632974,0.988949,0.663512,0.352747,0.504031,0.959438,0.748578,0.741053,0.615170,0.872494,0.847826
2023-01-08,0.665970,0.626657,1.133667,0.335967,0.400830,0.616297,0.670191,0.979796,0.684515,0.351273,0.465599,0.918960,0.733499,0.756931,0.719661,0.859242,0.825650
2023-01-15,0.688264,0.592654,1.112416,0.326186,0.395299,0.612507,0.659149,0.954521,0.665603,0.362438,0.433410,0.882806,0.697778,0.717173,0.666800,0.852531,0.819714


In [127]:
volatility_df.to_csv('test_vol.csv')

In [8]:
import numpy as np



# Replace infinities with NaN
# returns.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop NaN values
volatility_df.dropna(inplace=True)

# Or fill forward
# volatility_df.fillna(method='ffill', inplace=True)

# Or fill backward
# volatility_df.fillna(method='bfill', inplace=True)

# Or interpolate
volatility_df.interpolate(method='linear', inplace=True)


In [88]:
volatility_df

,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN,DIS,MCD,NEE,BA,CAT,XOM,CVX,RIO,BHP
Date,,,,,,,,,,,,,,,,,
2009-02-08,0.721723,0.818089,1.462633,0.395512,0.516632,1.948434,1.180200,1.014781,0.784249,0.448820,0.596133,1.013027,1.194089,0.617253,0.705557,1.755550,1.130694
2009-02-15,0.800277,0.756112,1.424457,0.386939,0.497923,1.821259,1.247614,1.015741,0.751184,0.402952,0.533393,0.991368,1.158939,0.586355,0.661857,1.743420,1.130840
2009-02-22,0.789265,0.903222,1.394273,0.379074,0.500720,1.590158,1.117247,0.989905,0.725258,0.396913,0.545094,0.944989,1.174691,0.557170,0.621621,1.689012,1.119671
2009-03-01,0.730268,0.831123,1.360457,0.373582,0.546323,1.388143,1.070177,1.082140,0.679356,0.373350,0.652686,0.899080,1.273301,0.540053,0.580762,1.632906,1.086366
2009-03-08,0.805162,0.986085,1.517000,0.354985,0.523159,1.285763,1.370726,1.123430,0.694543,0.361544,0.580062,0.862103,1.217686,0.578771,0.645352,2.084422,1.274090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-01,0.667170,0.635713,1.160935,0.335674,0.400336,0.616628,0.632974,0.988949,0.663533,0.354889,0.504040,0.959438,0.748667,0.741053,0.615190,0.866445,0.847604
2023-01-08,0.665971,0.626790,1.133667,0.335967,0.394225,0.613662,0.670191,0.979796,0.684531,0.353236,0.465607,0.918960,0.733583,0.756931,0.719683,0.853072,0.825430
2023-01-15,0.688265,0.592801,1.112416,0.326186,0.388962,0.610813,0.659149,0.954521,0.665617,0.365397,0.433419,0.882806,0.697857,0.717173,0.666820,0.846313,0.819580


In [89]:
mu = expected_returns.mean_historical_return(data_window)
S =risk_models.sample_cov(data_window)


In [90]:
S =  risk_models.sample_cov(data_window)
# S =np.cov(data_window)
S_adj =  S.copy()
for symbols in stocks:
    S_adj[symbols][symbols] = volatility_df[volatility_df.index == date][symbols]**(2) #Volatility = Variance^1/2 = Standard Deviation

ValueError: No axis named 1 for object type Series

In [ ]:
S_adj

,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN,DIS,MCD,NEE,BA,CAT,XOM,CVX,RIO,BHP
AAPL,0.070967,0.018206,0.019168,0.010635,0.010151,0.015689,0.015532,0.018577,0.013300,0.010370,0.006528,0.016455,0.016884,0.012073,0.012828,0.016721,0.017755
MSFT,0.018206,0.067827,0.025893,0.013602,0.012105,0.021510,0.020994,0.027030,0.017956,0.014579,0.011996,0.015645,0.019634,0.016352,0.017380,0.019693,0.024652
NVDA,0.019168,0.025893,0.174943,0.012013,0.012549,0.023663,0.023516,0.023010,0.021142,0.012394,0.009928,0.016939,0.017510,0.017696,0.018298,0.020796,0.020378
JNJ,0.010635,0.013602,0.012013,0.028895,0.010566,0.015462,0.015572,0.015778,0.013759,0.012092,0.011097,0.014879,0.011858,0.013271,0.013140,0.012845,0.014146
NVS,0.010151,0.012105,0.012549,0.010566,0.033728,0.012448,0.013018,0.012610,0.013560,0.008817,0.009904,0.011366,0.008999,0.010747,0.011320,0.016020,0.016725
JPM,0.015689,0.021510,0.023663,0.015462,0.012448,0.068905,0.031055,0.022781,0.019888,0.013637,0.011010,0.019604,0.018901,0.019105,0.020044,0.021848,0.021244
GS,0.015532,0.020994,0.023516,0.015572,0.013018,0.031055,0.072584,0.024616,0.020239,0.013608,0.010162,0.020024,0.019901,0.018880,0.019726,0.022448,0.021558
AMZN,0.018577,0.027030,0.023010,0.015778,0.012610,0.022781,0.024616,0.078948,0.023871,0.015277,0.009997,0.021376,0.018288,0.015854,0.017691,0.020650,0.025167
DIS,0.013300,0.017956,0.021142,0.013759,0.013560,0.019888,0.020239,0.023871,0.058918,0.013231,0.010704,0.018475,0.013951,0.014461,0.014461,0.017491,0.017818
MCD,0.010370,0.014579,0.012394,0.012092,0.008817,0.013637,0.013608,0.015277,0.013231,0.035925,0.009396,0.012998,0.012230,0.012304,0.011611,0.013629,0.014545


In [ ]:
S 

,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN,DIS,MCD,NEE,BA,CAT,XOM,CVX,RIO,BHP
AAPL,0.058274,0.018206,0.019168,0.010635,0.010151,0.015689,0.015532,0.018577,0.013300,0.010370,0.006528,0.016455,0.016884,0.012073,0.012828,0.016721,0.017755
MSFT,0.018206,0.062191,0.025893,0.013602,0.012105,0.021510,0.020994,0.027030,0.017956,0.014579,0.011996,0.015645,0.019634,0.016352,0.017380,0.019693,0.024652
NVDA,0.019168,0.025893,0.080974,0.012013,0.012549,0.023663,0.023516,0.023010,0.021142,0.012394,0.009928,0.016939,0.017510,0.017696,0.018298,0.020796,0.020378
JNJ,0.010635,0.013602,0.012013,0.022835,0.010566,0.015462,0.015572,0.015778,0.013759,0.012092,0.011097,0.014879,0.011858,0.013271,0.013140,0.012845,0.014146
NVS,0.010151,0.012105,0.012549,0.010566,0.026887,0.012448,0.013018,0.012610,0.013560,0.008817,0.009904,0.011366,0.008999,0.010747,0.011320,0.016020,0.016725
JPM,0.015689,0.021510,0.023663,0.015462,0.012448,0.038988,0.031055,0.022781,0.019888,0.013637,0.011010,0.019604,0.018901,0.019105,0.020044,0.021848,0.021244
GS,0.015532,0.020994,0.023516,0.015572,0.013018,0.031055,0.038817,0.024616,0.020239,0.013608,0.010162,0.020024,0.019901,0.018880,0.019726,0.022448,0.021558
AMZN,0.018577,0.027030,0.023010,0.015778,0.012610,0.022781,0.024616,0.098391,0.023871,0.015277,0.009997,0.021376,0.018288,0.015854,0.017691,0.020650,0.025167
DIS,0.013300,0.017956,0.021142,0.013759,0.013560,0.019888,0.020239,0.023871,0.038648,0.013231,0.010704,0.018475,0.013951,0.014461,0.014461,0.017491,0.017818
MCD,0.010370,0.014579,0.012394,0.012092,0.008817,0.013637,0.013608,0.015277,0.013231,0.024089,0.009396,0.012998,0.012230,0.012304,0.011611,0.013629,0.014545


In [ ]:
ef = EfficientFrontier(mu, S_adj, weight_bounds=(0, 1))  # Set bounds to allow shorting
raw_weights = ef.max_sharpe()  # Optimize for maximal Sharpe ratio
cleaned_weights = ef.clean_weights()  #

We learned of annulaized volatility, that the riskmodels.sample_cov assumes. 
I need to set it to monthly now, as well as the forecasted volatilities.
i need to check how that rebalancing loop works, i think its messing up.

In [ ]:
# Set the number of assets, assumed that 'df' is your DataFrame with historical prices.
df = stock_data.copy()
num_assets = len(df.columns)
stocks = df.columns

# Cred portfolio weights over time
optimal_weights_df = pd.DataFrame(index=df.index, columns=stocks)

# Set the window length for the rolling covariance calculation
window_length = 252  # Use 252 for approx one year of trading days

# Set the initial weights to be equal for all assets
initial_weights = np.array([1/num_assets]*num_assets)

for end_date in df.index[window_length:]:
    
    # Extract the window of data
    data_window = df.loc[:end_date].tail(window_length)
    
    # Calculate expected returns and covariance matrix for the window
    mu = expected_returns.mean_historical_return(data_window)

    # Get the date of the next period
    future_dates = df.index[df.index > end_date]
    if len(future_dates) > 0:
        next_period_date = future_dates[0]
    else:
        break
    # Get the date of the next period
    next_period_date = df.index[df.index > end_date][0]
    
    # Get the date of the closest week in the std_data dataframe
    closest_week_date = std_data.index[std_data.index.get_loc(next_period_date, method='nearest')]

    # Get the volatility data for the closest week
    std_data_next = std_data.loc[closest_week_date]

    
    # replace diagonal elements of cov matrix by GARCH-predicted variance.

    S =  risk_models.sample_cov(data_window)
    S_adj =  S.copy()
    for symbols in stocks:
        S_adj[symbols][symbols] = std_data_next[symbols]**(1/2) #Volatility = Variance^1/2 = Standard Deviation
    
    # Initialise and optimize the portfolio on the window of data
    ef = EfficientFrontier(mu, S_adj, weight_bounds=(0, 1))  # Set bounds to allow shorting
    raw_weights = ef.max_sharpe()  # Optimize for maximal Sharpe ratio
    cleaned_weights = ef.clean_weights()  # Clean the raw weights
    
    # Add the weights to our optimal_weights dataframe
    optimal_weights_df.loc[end_date] = [cleaned_weights.get(stock) for stock in stocks]

# Forward fill the weights for the days we aren't rebalancing
optimal_weights_df = optimal_weights_df.fillna(method='ffill')

# Print out the final DataFrame
print(optimal_weights_df.tail())

In [ ]:
stocks = ['AAPL', 'MSFT', 'NVDA', 'JNJ', 'NVS','JPM','GS','AMZN','DIS','MCD','NEE','BA','CAT','XOM','CVX','RIO','BHP']

start_date = '2009-01-01' 

stock_data = yf.download(stocks, start=start_date)['Close']
stock_data = stock_data.dropna()
stock_data = stock_data.reindex(columns=stocks)
stock_prices = stock_data[stocks].values

[*********************100%***********************]  17 of 17 completed


In [ ]:
df = stock_data.copy()
eom_dates = volatility_df.resample('M').last().index
for end_date in eom_dates[eom_dates >= df.index[window_length]]:
    print(end_date)

In [ ]:
vol_data_ind = volatility_df.resample('M').last()
window_length = 31
vol_data_ind.index[window_length:]

DatetimeIndex(['2011-09-30', '2011-10-31', '2011-11-30', '2011-12-31',
               '2012-01-31', '2012-02-29', '2012-03-31', '2012-04-30',
               '2012-05-31', '2012-06-30',
               ...
               '2022-04-30', '2022-05-31', '2022-06-30', '2022-07-31',
               '2022-08-31', '2022-09-30', '2022-10-31', '2022-11-30',
               '2022-12-31', '2023-01-31'],
              dtype='datetime64[ns]', name='Date', length=137, freq='M')

In [ ]:
volatility_df_month = volatility_df.resample('M').last()

In [96]:
volatility_df

,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN,DIS,MCD,NEE,BA,CAT,XOM,CVX,RIO,BHP
Date,,,,,,,,,,,,,,,,,
2009-02-08,0.721723,0.818089,1.462633,0.395512,0.516632,1.948434,1.180200,1.014781,0.784249,0.448820,0.596133,1.013027,1.194089,0.617253,0.705557,1.755550,1.130694
2009-02-15,0.800277,0.756112,1.424457,0.386939,0.497923,1.821259,1.247614,1.015741,0.751184,0.402952,0.533393,0.991368,1.158939,0.586355,0.661857,1.743420,1.130840
2009-02-22,0.789265,0.903222,1.394273,0.379074,0.500720,1.590158,1.117247,0.989905,0.725258,0.396913,0.545094,0.944989,1.174691,0.557170,0.621621,1.689012,1.119671
2009-03-01,0.730268,0.831123,1.360457,0.373582,0.546323,1.388143,1.070177,1.082140,0.679356,0.373350,0.652686,0.899080,1.273301,0.540053,0.580762,1.632906,1.086366
2009-03-08,0.805162,0.986085,1.517000,0.354985,0.523159,1.285763,1.370726,1.123430,0.694543,0.361544,0.580062,0.862103,1.217686,0.578771,0.645352,2.084422,1.274090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-01,0.667170,0.635713,1.160935,0.335674,0.400336,0.616628,0.632974,0.988949,0.663533,0.354889,0.504040,0.959438,0.748667,0.741053,0.615190,0.866445,0.847604
2023-01-08,0.665971,0.626790,1.133667,0.335967,0.394225,0.613662,0.670191,0.979796,0.684531,0.353236,0.465607,0.918960,0.733583,0.756931,0.719683,0.853072,0.825430
2023-01-15,0.688265,0.592801,1.112416,0.326186,0.388962,0.610813,0.659149,0.954521,0.665617,0.365397,0.433419,0.882806,0.697857,0.717173,0.666820,0.846313,0.819580


In [ ]:
volatility_df

,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN,DIS,MCD,NEE,BA,CAT,XOM,CVX,RIO,BHP
Date,,,,,,,,,,,,,,,,,
2009-02-08,0.721723,0.818089,1.462633,0.395512,0.516632,1.948434,1.180200,1.014781,0.784249,0.448820,0.596133,1.013027,1.194089,0.617253,0.705557,1.755550,1.130694
2009-02-15,0.800277,0.756112,1.424457,0.386939,0.497923,1.821259,1.247614,1.015741,0.751184,0.402952,0.533393,0.991368,1.158939,0.586355,0.661857,1.743420,1.130840
2009-02-22,0.789265,0.903222,1.394273,0.379074,0.500720,1.590158,1.117247,0.989905,0.725258,0.396913,0.545094,0.944989,1.174691,0.557170,0.621621,1.689012,1.119671
2009-03-01,0.730268,0.831123,1.360457,0.373582,0.546323,1.388143,1.070177,1.082140,0.679356,0.373350,0.652686,0.899080,1.273301,0.540053,0.580762,1.632906,1.086366
2009-03-08,0.805162,0.986085,1.517000,0.354985,0.523159,1.285763,1.370726,1.123430,0.694543,0.361544,0.580062,0.862103,1.217686,0.578771,0.645352,2.084422,1.274090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-01,0.667170,0.635713,1.160935,0.335674,0.400336,0.616628,0.632974,0.988949,0.663533,0.354889,0.504040,0.959438,0.748667,0.741053,0.615190,0.866445,0.847604
2023-01-08,0.665971,0.626790,1.133667,0.335967,0.394225,0.613662,0.670191,0.979796,0.684531,0.353236,0.465607,0.918960,0.733583,0.756931,0.719683,0.853072,0.825430
2023-01-15,0.688265,0.592801,1.112416,0.326186,0.388962,0.610813,0.659149,0.954521,0.665617,0.365397,0.433419,0.882806,0.697857,0.717173,0.666820,0.846313,0.819580


In [102]:
stock_data

,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN,DIS,MCD,NEE,BA,CAT,XOM,CVX,RIO,BHP
Date,,,,,,,,,,,,,,,,,
2009-01-02,3.241071,20.330000,2.177500,60.650002,44.390682,31.350000,86.760002,2.718000,23.920000,63.750000,12.915000,45.250000,46.910000,81.639999,76.519997,24.850000,40.597679
2009-01-05,3.377857,20.520000,2.217500,60.049999,43.888889,29.250000,88.779999,2.703000,23.500000,63.560001,12.985000,46.169998,46.080002,81.629997,76.660004,25.387501,40.642284
2009-01-06,3.322143,20.760000,2.292500,59.689999,43.189964,29.879999,88.709999,2.868000,24.309999,62.139999,12.802500,46.310001,45.799999,80.300003,77.349998,29.299999,42.818913
2009-01-07,3.250357,19.510000,2.155000,59.130001,43.637993,28.090000,84.500000,2.810000,23.180000,61.240002,12.662500,44.759998,43.669998,78.250000,73.959999,27.100000,39.607494
2009-01-08,3.310714,20.120001,2.100000,59.020000,44.462364,27.219999,85.410004,2.858000,22.900000,60.520000,12.775000,44.790001,44.020000,79.089996,74.239998,27.117500,39.973240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-28,189.250000,335.850006,411.170013,162.960007,98.440002,138.589996,313.660004,129.039993,88.830002,291.739990,73.440002,210.720001,241.869995,105.400002,154.919998,63.430000,59.380001
2023-06-29,189.589996,335.049988,408.220001,164.100006,99.360001,143.429993,323.089996,127.900002,88.949997,294.470001,72.809998,211.830002,244.240005,106.699997,156.240005,63.799999,59.770000
2023-06-30,193.970001,340.540009,423.019989,165.520004,100.910004,145.440002,322.540009,130.360001,89.279999,298.410004,74.199997,211.160004,246.050003,107.250000,157.350006,63.840000,59.669998


In [103]:

# Drop NaN values
volatility_df.dropna(inplace=True)

# Or fill forward
# volatility_df.fillna(method='ffill', inplace=True)

# Or fill backward
# volatility_df.fillna(method='bfill', inplace=True)

# Or interpolate
volatility_df.interpolate(method='linear', inplace=True)


In [9]:
stocks = ['AAPL', 'MSFT', 'NVDA', 'JNJ', 'NVS','JPM','GS','AMZN','DIS','MCD','NEE','BA','CAT','XOM','CVX','RIO','BHP']
stock_data = yf.download(stocks, start=start_date)['Adj Close']
stock_data = stock_data.dropna()
stock_data = stock_data.reindex(columns=stocks)

[*********************100%***********************]  17 of 17 completed


In [15]:
# Set the number of assets, assumed that 'df' is your DataFrame with historical prices.
df = stock_data.copy()
num_assets = len(df.columns)
stocks = df.columns
window_length = 31 #months

vol_data_ind = volatility_df.resample('M').last().index
volatility_df_month = volatility_df.resample('M').last()


# Cred portfolio weights over time
optimal_weights_df = pd.DataFrame(index=df.index, columns=stocks)

# Loop over each end-of-month date
for end_date in vol_data_ind[window_length:]:
    
    # Extract the window of data
    data_window = df.loc[:end_date].tail(window_length*620)
    
    #Check if we at the end of the month, if this end date is within 2 days of :

    # Calculate expected returns and covariance matrix for the window, these are mean daily returns
    mu = expected_returns.mean_historical_return(data_window)
    
    # Get the dates that are greater than end_date
    future_dates = vol_data_ind[vol_data_ind > end_date]

    # Check if there are any future dates
    if len(future_dates) > 0:
        # Get the first future date
        next_month = future_dates[0]
    else:
        # If there are no future dates, break the loop
        break
    
    next_month = vol_data_ind[vol_data_ind > end_date][0]
    # Get the volatility data for the closest week
    volatility_df_pred = volatility_df_month[volatility_df_month.index == next_month]

    # replace diagonal elements of cov matrix by GARCH-predicted variance.
    S =  risk_models.sample_cov(data_window)
    S_adj =  S.copy()
    for symbols in stocks:
        S_adj[symbols][symbols] = volatility_df_pred[symbols][0]**(2) #Volatility = Variance^1/2 = Standard Deviation
    
    # Initialise and optimize the portfolio on the window of data
    ef = EfficientFrontier(mu, S_adj, weight_bounds=(0, 1))  # Set bounds to allow shorting
    raw_weights = ef.max_sharpe()  # Optimize for maximal Sharpe ratio
    cleaned_weights = ef.clean_weights()  # Clean the raw weights
    
    # Add the weights to our optimal_weights dataframe
    optimal_weights_df.loc[end_date] = [cleaned_weights.get(stock) for stock in stocks]

# Forward fill the weights for the days we aren't rebalancing
optimal_weights_df = optimal_weights_df.fillna(method='ffill')

# Print out the final DataFrame
print(optimal_weights_df.tail())


               AAPL     MSFT     NVDA      JNJ      NVS      JPM   GS  \
Date                                                                    
2021-07-31  0.18555  0.14085  0.07823  0.08476  0.01526  0.00000  0.0   
2021-10-31  0.19487  0.11306  0.09183  0.07532  0.00000  0.00372  0.0   
2022-04-30  0.21496  0.13932  0.06153  0.16683  0.03596  0.00000  0.0   
2022-07-31  0.21276  0.11034  0.06247  0.13599  0.04204  0.00226  0.0   
2022-12-31  0.16678  0.11042  0.05324  0.13905  0.03809  0.00650  0.0   

               AMZN      DIS      MCD      NEE   BA      CAT  XOM  CVX  \
Date                                                                     
2021-07-31  0.13706  0.06455  0.10135  0.19034  0.0  0.00205  0.0  0.0   
2021-10-31  0.17777  0.05532  0.09909  0.18773  0.0  0.00130  0.0  0.0   
2022-04-30  0.09600  0.01759  0.17705  0.05500  0.0  0.02420  0.0  0.0   
2022-07-31  0.09197  0.02233  0.17820  0.11887  0.0  0.01507  0.0  0.0   
2022-12-31  0.06132  0.00000  0.16194  0.220

In [16]:
optimal_weights_df.sort_index()

,AAPL,MSFT,NVDA,JNJ,NVS,JPM,GS,AMZN,DIS,MCD,NEE,BA,CAT,XOM,CVX,RIO,BHP
Date,,,,,,,,,,,,,,,,,
2009-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-28,0.16773,0.09443,0.05977,0.1482,0.04868,0.0158,0.00914,0.0633,0.01063,0.21248,0.12369,0.00067,0.02529,0.0,0.00677,0.01342,0.0
2023-06-29,0.16773,0.09443,0.05977,0.1482,0.04868,0.0158,0.00914,0.0633,0.01063,0.21248,0.12369,0.00067,0.02529,0.0,0.00677,0.01342,0.0
2023-06-30,0.16773,0.09443,0.05977,0.1482,0.04868,0.0158,0.00914,0.0633,0.01063,0.21248,0.12369,0.00067,0.02529,0.0,0.00677,0.01342,0.0
